# No-KYC Visa Card Monitor Dashboard

This dashboard visualizes data collected by the No-KYC Card Monitor, which scans the web daily for prepaid Visa cards that can be obtained without KYC (Know Your Customer) verification.

**Data Source:** Automated daily scans from Google Search, Reddit, App Store, Google Play, LinkedIn, and other sources.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [ ]:
# Fetch live data from the repository
csv_url = "https://raw.githubusercontent.com/0xSector/-no-kyc-card-monitor/main/output/all_results.csv"

try:
    df = pd.read_csv(csv_url)
    print(f"Loaded {len(df)} records from GitHub")
except Exception as e:
    print(f"Error loading data: {e}")
    # Fallback: create empty DataFrame with expected columns
    df = pd.DataFrame(columns=['date_found', 'source_platform', 'card_name', 'card_type', 
                               'company_name', 'company_website', 'issuing_bank', 'notes'])

In [ ]:
# Data preparation
df['date_found'] = pd.to_datetime(df['date_found'], errors='coerce')

# Extract platform category from source_platform
def categorize_platform(platform):
    if pd.isna(platform):
        return 'Unknown'
    platform = str(platform).lower()
    if 'reddit' in platform:
        return 'Reddit'
    elif 'app store' in platform:
        return 'App Store'
    elif 'google play' in platform:
        return 'Google Play'
    elif 'linkedin' in platform:
        return 'LinkedIn'
    elif 'x/twitter' in platform or 'twitter' in platform:
        return 'X/Twitter'
    elif 'bitcointalk' in platform:
        return 'BitcoinTalk'
    elif 'youtube' in platform:
        return 'YouTube'
    elif 'web' in platform:
        return 'Web'
    else:
        return 'Other'

df['platform_category'] = df['source_platform'].apply(categorize_platform)

# Clean card types
def clean_card_type(card_type):
    if pd.isna(card_type):
        return 'Unknown'
    card_type = str(card_type)
    if 'virtual' in card_type.lower():
        return 'Virtual'
    elif 'prepaid' in card_type.lower():
        return 'Prepaid'
    elif 'debit' in card_type.lower():
        return 'Debit'
    elif 'credit' in card_type.lower():
        return 'Credit'
    else:
        return 'Other'

df['card_type_clean'] = df['card_type'].apply(clean_card_type)

print(f"Date range: {df['date_found'].min()} to {df['date_found'].max()}")
print(f"Unique platforms: {df['platform_category'].nunique()}")
print(f"Unique card types: {df['card_type_clean'].nunique()}")

## Summary Metrics

In [ ]:
# Key metrics
total_cards = len(df)
unique_companies = df['company_name'].dropna().nunique()
unique_websites = df['company_website'].dropna().nunique()
cards_with_issuing_bank = df['issuing_bank'].notna().sum()
days_tracked = (df['date_found'].max() - df['date_found'].min()).days + 1 if len(df) > 0 else 0

print(f"{'='*50}")
print(f"NO-KYC CARD MONITOR SUMMARY")
print(f"{'='*50}")
print(f"Total Cards Found:        {total_cards:>10}")
print(f"Unique Companies:         {unique_companies:>10}")
print(f"Unique Websites:          {unique_websites:>10}")
print(f"With Issuing Bank Info:   {cards_with_issuing_bank:>10}")
print(f"Days Tracked:             {days_tracked:>10}")
print(f"{'='*50}")

## Cards Found Over Time

In [ ]:
# Daily cards found
daily_counts = df.groupby('date_found').size().reset_index(name='count')

fig = px.bar(
    daily_counts,
    x='date_found',
    y='count',
    title='Cards Discovered Per Day',
    labels={'date_found': 'Date', 'count': 'Cards Found'},
    color_discrete_sequence=['#1a365d']
)
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Number of Cards',
    showlegend=False
)
fig.show()

## Source Platform Distribution

In [ ]:
# Platform distribution
platform_counts = df['platform_category'].value_counts().reset_index()
platform_counts.columns = ['Platform', 'Count']

fig = px.pie(
    platform_counts,
    values='Count',
    names='Platform',
    title='Cards by Source Platform',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## Card Type Breakdown

In [ ]:
# Card type distribution
card_type_counts = df['card_type_clean'].value_counts().reset_index()
card_type_counts.columns = ['Card Type', 'Count']

fig = px.bar(
    card_type_counts,
    x='Card Type',
    y='Count',
    title='Distribution by Card Type',
    color='Card Type',
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(showlegend=False)
fig.show()

## Platform Trends Over Time

In [ ]:
# Platform trends over time
platform_daily = df.groupby(['date_found', 'platform_category']).size().reset_index(name='count')

fig = px.area(
    platform_daily,
    x='date_found',
    y='count',
    color='platform_category',
    title='Source Platform Trends Over Time',
    labels={'date_found': 'Date', 'count': 'Cards Found', 'platform_category': 'Platform'}
)
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Number of Cards',
    legend_title='Platform'
)
fig.show()

## Top Companies by Card Mentions

In [ ]:
# Top companies
company_counts = df['company_name'].dropna().value_counts().head(15).reset_index()
company_counts.columns = ['Company', 'Mentions']

fig = px.bar(
    company_counts,
    x='Mentions',
    y='Company',
    orientation='h',
    title='Top 15 Companies by Card Mentions',
    color='Mentions',
    color_continuous_scale='Blues'
)
fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    showlegend=False
)
fig.show()

## Issuing Banks

In [ ]:
# Issuing banks (where available)
bank_df = df[df['issuing_bank'].notna() & (df['issuing_bank'] != '')]

if len(bank_df) > 0:
    bank_counts = bank_df['issuing_bank'].value_counts().head(10).reset_index()
    bank_counts.columns = ['Issuing Bank', 'Count']
    
    fig = px.bar(
        bank_counts,
        x='Count',
        y='Issuing Bank',
        orientation='h',
        title='Top Issuing Banks',
        color='Count',
        color_continuous_scale='Greens'
    )
    fig.update_layout(
        yaxis={'categoryorder': 'total ascending'},
        showlegend=False
    )
    fig.show()
else:
    print("No issuing bank data available yet.")

## Data Quality Overview

In [ ]:
# Data completeness
fields = ['card_name', 'company_name', 'company_website', 'issuing_bank', 
          'terms_conditions_url', 'privacy_policy_url', 'contact_email']

completeness = []
for field in fields:
    if field in df.columns:
        filled = df[field].notna().sum()
        pct = (filled / len(df)) * 100 if len(df) > 0 else 0
        completeness.append({'Field': field, 'Filled': filled, 'Percentage': pct})

completeness_df = pd.DataFrame(completeness)

fig = px.bar(
    completeness_df,
    x='Field',
    y='Percentage',
    title='Data Completeness by Field (%)',
    color='Percentage',
    color_continuous_scale='RdYlGn',
    range_color=[0, 100]
)
fig.update_layout(
    xaxis_title='Field',
    yaxis_title='% Complete',
    yaxis_range=[0, 100]
)
fig.show()

## Recent Discoveries

In [ ]:
# Most recent cards found
recent_cols = ['date_found', 'card_name', 'company_name', 'card_type', 'platform_category']
recent_df = df.sort_values('date_found', ascending=False).head(20)[recent_cols]

print("20 Most Recent Card Discoveries")
print("=" * 80)
recent_df

## Full Data Table

In [ ]:
# Display full dataset with key columns
display_cols = ['date_found', 'card_name', 'company_name', 'company_website', 
                'card_type', 'issuing_bank', 'platform_category']
df[display_cols].sort_values('date_found', ascending=False)